Importing useful libraries 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import os
#print(os.getcwd())


Loading data from file 

In [2]:
data = pd.read_csv('~/Downloads/multi_classification_train(1).csv')
data.head()

,ID,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18,Feature_19,Feature_20,Class
0,1,-90.532634,-66.953472,79.261856,-111.800554,126.403549,111.509348,207.728953,-26.600556,-106.229324,...,148.391734,77.835738,-10.728207,1.100625,1.035362,-87.010809,-16.242297,-32.074925,-17.666026,1
1,2,-103.156720,-16.070400,87.819228,12.614599,51.347780,67.483725,40.269172,51.442254,-151.486693,...,29.009475,3.995786,-10.861630,-142.605726,-25.924592,-86.755351,-36.479749,-130.246619,-44.143652,2
2,3,24.326153,-92.098078,82.238354,-56.795879,85.203996,127.916504,-90.080307,-128.124071,18.036020,...,111.810098,65.826018,-101.271203,-44.127749,-7.131464,-105.049759,-130.948256,-43.113523,-37.330448,4
3,4,-64.631737,-83.703583,84.135072,-5.516152,74.338494,112.630556,181.576798,-1.054023,60.469865,...,50.047108,100.439101,-117.842955,150.239788,-144.635542,-144.306209,-69.272905,-79.629675,-51.334456,2
4,5,-55.473830,-78.853237,88.129107,75.200543,76.991520,60.224711,-13.106559,-146.773016,-33.490566,...,85.988282,23.381960,11.876102,-188.296503,-80.323929,-56.757987,-20.314172,-42.625170,-24.102753,2


Extracting and normalising the data

In [3]:
x = data.iloc[:,1:21]
print(x.head())
x = (x - x.mean())/x.std()
print(x.head())
x = x.values
print(x.shape)
y = data.iloc[:,21].values
print(y.shape)
print(type(y))

    Feature_1  Feature_2  Feature_3   Feature_4   Feature_5   Feature_6  \
0  -90.532634 -66.953472  79.261856 -111.800554  126.403549  111.509348   
1 -103.156720 -16.070400  87.819228   12.614599   51.347780   67.483725   
2   24.326153 -92.098078  82.238354  -56.795879   85.203996  127.916504   
3  -64.631737 -83.703583  84.135072   -5.516152   74.338494  112.630556   
4  -55.473830 -78.853237  88.129107   75.200543   76.991520   60.224711   

    Feature_7   Feature_8   Feature_9  Feature_10  Feature_11  Feature_12  \
0  207.728953  -26.600556 -106.229324  170.384649   69.038320  148.391734   
1   40.269172   51.442254 -151.486693   35.872507   28.016774   29.009475   
2  -90.080307 -128.124071   18.036020   70.485076   56.468318  111.810098   
3  181.576798   -1.054023   60.469865   -0.658352   35.245623   50.047108   
4  -13.106559 -146.773016  -33.490566  137.684666   47.595552   85.988282   

   Feature_13  Feature_14  Feature_15  Feature_16  Feature_17  Feature_18  \
0   77.83

Defining one hot encoding functions

In [4]:

# .values se ek column wala numpy array nhi bnta
def one_hot_encode(arr):
    

    unique_values = np.unique(arr) # gibr a list of unique elements in arr
    
   
    value_to_index = {val: idx for idx, val in enumerate(unique_values)} #  mapping of unique values to indices
    
  
    one_hot_matrix = np.zeros((len(arr), len(unique_values)), dtype = np.int64)        # Initialize a zero matrix of shape (len(arr), len(unique_values))
    
    
    for i, val in enumerate(arr):
        one_hot_matrix[i, value_to_index[val]] = 1    # Setting the appropriate indices to 1
    
    return one_hot_matrix


spilliting data for cross validation and taking transpose 

In [5]:


print((y.shape))
m = x.shape[0]

n = int(0.2*m)
x_test = x[:n,:].T
x_train = x[n:,:].T
print(x_train.shape)

y_test = y[:n].T
y_train_not_one = y[n:]
print(y_test.shape)
print(y_test)

(48000,)
(20, 38400)
(9600,)
[1 2 4 ... 3 4 2]


one hot encoding of training output

In [6]:
print(y_train_not_one)
a = np.unique(y_train_not_one)
print(a)
print(y_train_not_one.shape)
class_counts = pd.Series(y).value_counts(normalize = True)  # normalize = true krne pr proprtion deta hai 
print(class_counts) 
y_train = one_hot_encode(y_train_not_one).T
print(y_train.shape)
print(y_train)
print(y_train.shape)

print(y_test)


[2 3 2 ... 1 3 2]
[0 1 2 3 4]
(38400,)
2    0.346208
1    0.237583
3    0.209667
4    0.122375
0    0.084167
Name: proportion, dtype: float64
(5, 38400)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [1 0 1 ... 0 0 1]
 [0 1 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]]
(5, 38400)
[1 2 4 ... 3 4 2]


Defining functions 

In [7]:
def cost_function(a3,y,l,w1,w2,w3):
    
    e = 1e-10  # for safety 
    m = y.shape[1]
    cost  = -(1/m)*(np.sum(y*np.log(a3 + e)))+ (l/(2*m))*(np.sum(np.square(w1))+ np.sum(np.square(w2)) + np.sum(np.square(w3)))
    return cost

In [8]:

n_x = x_train.shape[0]
n_h1 = 1024
n_h2 = 512
n_y = y_train.shape[0]
alpha = 0.01
iteration = 100
l = 0.0001

w1 = np.random.randn(n_h1,n_x)*np.sqrt(2/n_x)

b1 = np.zeros((n_h1, 1))

w2 = np.random.randn(n_h2,n_h1)*np.sqrt(2/n_h1)

b2 = np.zeros((n_h2,1))

w3 = np.random.randn(n_y,n_h2)*0.01

b3 = np.zeros((n_y,1))

def relu(x_train):
    return np.maximum(x_train, 0)
def softmax(x_train):
    expX = np.exp(x_train)
    return expX/np.sum(expX, axis=0)
    

def d_relu(x_train):
    return np.array(x_train > 0, dtype = np.float32)

In [15]:

def compute_accuracy(a3,y):
    a3_index = np.argmax(a3, axis = 0) 
    accuracy = (np.mean(a3_index == y))*100
    return accuracy

# Initialize Adam parameters
beta1 = 0.9  # Exponential decay for the first moment estimates
beta2 = 0.999  # Exponential decay for the second moment estimates
epsilon = 1e-8  # Small constant to avoid division by zero
alpha_initial = 0.001  # Learning rate
decay_rate = 0.96
decay_step = 100
l = 0.01
# Initialize moment estimates for all parameters
m_w1, m_w2, m_w3 = 0, 0, 0  # First moments for weights
v_w1, v_w2, v_w3 = 0, 0, 0  # Second moments for weights
m_b1, m_b2, m_b3 = 0, 0, 0  # First moments for biases
v_b1, v_b2, v_b3 = 0, 0, 0  # Second moments for biases

t = 0  # Time step for bias correction

j_history = []
for i in range(iteration):
    # Forward propagation
    z1 = np.dot(w1, x) + b1
    a1 = relu(z1)
    z2 = np.dot(w2, a1) + b2
    a2 = relu(z2)
    z3 = np.dot(w3, a2) + b3
    a3 = softmax(z3)
    
    # Backward propagation
    m = x.shape[1]
    dz3 = (a3 - y_train)
    dw3 = (1/m) * (np.dot(dz3, a2.T) + l * np.sum(w3))
    db3 = (1/m) * np.sum(dz3, axis=1, keepdims=True)
    dz2 = (1/m) * np.dot(w3.T, dz3) * d_relu(a2)
    dw2 = (1/m) * (np.dot(dz2, a1.T) + l * np.sum(w2))
    db2 = (1/m) * np.sum(dz2, axis=1, keepdims=True)
    dz1 = (1/m) * np.dot(w2.T, dz2) * d_relu(a1)
    dw1 = (1/m) * (np.dot(dz1, x.T) + l * np.sum(w1))
    db1 = (1/m) * np.sum(dz1, axis=1, keepdims=True)
    
    # Update Adam parameters
    t += 1
    alpha = alpha_initial * (decay_rate ** (t // decay_step))
    # Update weights and biases using Adam
    for param, grad, m, v in zip(
        [w1, w2, w3, b1, b2, b3],
        [dw1, dw2, dw3, db1, db2, db3],
        [m_w1, m_w2, m_w3, m_b1, m_b2, m_b3],
        [v_w1, v_w2, v_w3, v_b1, v_b2, v_b3],
    ):
        m = beta1 * m + (1 - beta1) * grad  # Update biased first moment
        v = beta2 * v + (1 - beta2) * (grad ** 2)  # Update biased second moment
        m_corrected = m / (1 - beta1 ** t)  # Correct bias for first moment
        v_corrected = v / (1 - beta2 ** t)  # Correct bias for second moment
        param -= alpha * m_corrected / (np.sqrt(v_corrected) + epsilon)  # Update parameter
    
    # Compute and store cost
    if i < iteration:
        c = cost_function(a3, y_train, l, w1, w2, w3)
        j_history.append(c)
        
    
    # Print cost at intervals
    if (i%(iteration/10) == 0):
        accuracy = compute_accuracy(a3, y_train_not_one)
        print(f'Iteration {i+1}, Cost: {c}, Accuracy: {accuracy}')

# Plot cost history
plt.plot(j_history)
plt.xlabel("Iteration")
plt.ylabel("Cost")
plt.title("Cost Reduction Over Iterations")
plt.show()


ValueError: shapes (1500,1024) and (80000,1024) not aligned: 1024 (dim 1) != 80000 (dim 0)

In [11]:
def prediction(x_test,w1,w2,w3,b1,b2,b3):
    z1 = np.dot(w1,x_test) + b1
    a1 = relu(z1)
    z2 = np.dot(w2,a1) + b2
    a2 = relu(z2)
    z3 = np.dot(w3,a2) + b3
    a3 = softmax(z3)
    

    print((a3.shape))

    a3_index = np.argmax(a3, axis = 0) 
    
    print(a3_index)
    return a3_index
a3_index = prediction(x_test,w1,w2,w3,b1,b2,b3)


class_counts = pd.Series(a3_index).value_counts(normalize = True)  # normalize = true krne pr proprtion deta hai 
print(class_counts) 
accuracy = np.mean(a3_index == y_test)
print(f"accuracy of cross validation set is {accuracy*100}")

a3_train_index = prediction(x_train,w1,w2,w3,b1,b2,b3)
class_counts = pd.Series(a3_train_index).value_counts(normalize = True)  
print(class_counts) 
print(a3_train_index)
print(y_train_not_one)
print(y_train_not_one.shape)
print(f"accuracy of training set is {np.mean(a3_train_index == y_train_not_one)*100}")

(5, 9600)
[1 2 4 ... 3 3 2]
2    0.353854
1    0.241354
3    0.210729
4    0.118438
0    0.075625
Name: proportion, dtype: float64
accuracy of cross validation set is 96.1875
(5, 38400)
[2 3 2 ... 1 3 2]
2    0.350859
1    0.237969
3    0.211068
4    0.120938
0    0.079167
Name: proportion, dtype: float64
[2 3 2 ... 1 3 2]
[2 3 2 ... 1 3 2]
(38400,)
accuracy of training set is 96.64583333333333


In [12]:


def f1_score_avg_weighted(y, y_test, n_classes):
    f1_scores = np.zeros(n_classes)
    class_count = np.zeros(n_classes)  
    e = 1e-9  
    
    for i in range(n_classes):  
        precision = np.sum((y == i) & (y_test == i)) / (np.sum(y == i) + e)
        recall = np.sum((y == i) & (y_test == i)) / (np.sum(y_test == i) + e)
        f1_scores[i] = 2 * (precision * recall) / (precision + recall + e)
        class_count[i] = np.sum((y == i) & (y_test == i))
    
    return (np.sum(f1_scores * class_count)/(np.sum(class_count) + e)) 


n_classes = len(np.unique(y_test))  
f1_avg = f1_score_avg_weighted(a3_index, y_test, n_classes)
print(f"Average F1 Score: {f1_avg}")


Average F1 Score: 0.9623253064306274


In [ ]:
data_test = pd.read_csv('~/Downloads/multi_classification_test.csv')
x_test = data_test.iloc[:,1:]
x_test = (x_test - x_test.mean())/(x_test.std())
x_test = x_test.values
y_pred = prediction(x_test,w1,w2,w3,b1,b2,b3) 
print(y_pred)
